In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install nltk

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import nltk
import string
from string import digits
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import tensorflow as tf
from nltk.translate.bleu_score import sentence_bleu
from keras.layers import Input, LSTM, Embedding, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import SmoothingFunction
from keras.models import Model
nltk.download('punkt')
print(os.listdir("."))

['.ipynb_checkpoints', 'ara_eng.txt', 'Eng2Ara_LSTM.ipynb', 'Eng2Ara_Transformer.ipynb', 'Group3.docx', 'X_test.pkl', 'X_train.pkl']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahran\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
batch_size = 64
epochs = 100

In [ ]:
file_path = "ara_eng.txt" # modify the file name and extension if needed

In [ ]:
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()


In [ ]:
# Load the text file into a Pandas DataFrame
lines = pd.read_table(file_path, header=None, names=['English_sentence', 'Arabic_sentence'])
lines.head()

,English_sentence,Arabic_sentence
0,Hi.,مرحبًا.
1,Run!,اركض!
2,Help!,النجدة!
3,Jump!,اقفز!
4,Stop!,قف!


In [ ]:
lines['English_sentence'].value_counts().sum()

24638

In [ ]:
# Lowercase all characters
lines['English_sentence']=lines['English_sentence'].apply(lambda x: x.lower())
lines['Arabic_sentence']=lines['Arabic_sentence'].apply(lambda x: x.lower())

In [ ]:
lines.head()

,English_sentence,Arabic_sentence
0,hi.,مرحبًا.
1,run!,اركض!
2,help!,النجدة!
3,jump!,اقفز!
4,stop!,قف!


In [ ]:
# Remove quotes
lines['English_sentence']=lines['English_sentence'].apply(lambda x: re.sub("'", '', x))
lines['Arabic_sentence']=lines['Arabic_sentence'].apply(lambda x: re.sub("'", '', x))

In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['English_sentence']=lines['English_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['Arabic_sentence']=lines['Arabic_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
lines.head()

,English_sentence,Arabic_sentence
0,hi,مرحبًا
1,run,اركض
2,help,النجدة
3,jump,اقفز
4,stop,قف


In [ ]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['English_sentence']=lines['English_sentence'].apply(lambda x: x.translate(remove_digits))
lines['Arabic_sentence'] = lines['Arabic_sentence'].apply(lambda x: re.sub("[٠١٢٣٤٥٦٧٨٩]", "", x))
# Remove extra spaces
lines['English_sentence']=lines['English_sentence'].apply(lambda x: x.strip())
lines['Arabic_sentence']=lines['Arabic_sentence'].apply(lambda x: x.strip())
lines['English_sentence']=lines['English_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['Arabic_sentence']=lines['Arabic_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [ ]:
# Add start and end tokens to target sequences
lines['Arabic_sentence'] = lines['Arabic_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines.head()

,English_sentence,Arabic_sentence
0,hi,START_ مرحبًا _END
1,run,START_ اركض _END
2,help,START_ النجدة _END
3,jump,START_ اقفز _END
4,stop,START_ قف _END


In [ ]:
### Get English and arabicVocabulary
all_english_words=set()
for eng in lines['English_sentence']:
    for word in eng.split():
        if word not in all_english_words:
            all_english_words.add(word)

all_arabic_words=set()
for ara in lines['Arabic_sentence']:
    for word in ara.split():
        if word not in all_arabic_words:
            all_arabic_words.add(word)

In [ ]:
len(all_english_words)

25996

In [ ]:
len(all_arabic_words)

57726

In [ ]:
lines['length_eng_sentence']=lines['English_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_ara_sentence']=lines['Arabic_sentence'].apply(lambda x:len(x.split(" ")))

In [ ]:
lines.head()

,English_sentence,Arabic_sentence,length_eng_sentence,length_ara_sentence
0,hi,START_ مرحبًا _END,1,3
1,run,START_ اركض _END,1,3
2,help,START_ النجدة _END,1,3
3,jump,START_ اقفز _END,1,3
4,stop,START_ قف _END,1,3


In [ ]:
lines[lines['length_eng_sentence']>50].shape

(2111, 4)

In [ ]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_ara_sentence']<=20]

In [ ]:
lines.shape

(17561, 4)

Perform a sanity check

In [ ]:
print("maximum length of Arabic Sentence ",max(lines['length_ara_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Arabic Sentence  20
maximum length of English Sentence  20


In [ ]:
max_length_eng=max(lines['length_eng_sentence'])
max_length_ara=max(lines['length_ara_sentence'])


In [ ]:
input_words = sorted(list(all_english_words))
target_words = sorted(list(all_arabic_words))
num_encoder_tokens = len(all_english_words)
num_decoder_tokens = len(all_arabic_words)
num_encoder_tokens, num_decoder_tokens

(25996, 57726)

In [ ]:
num_decoder_tokens += 1 #for zero padding

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines = shuffle(lines)
lines.head(10)

,English_sentence,Arabic_sentence,length_eng_sentence,length_ara_sentence
19330,exchange student starved in egypt global voices,START_ مصر تجويع طالب تبادل طلابي الاصوات العا...,7,9
11167,iran we re going to the world cup videos globa...,START_ مقاطع فيديو من ايران ذاهبون الى كاس الع...,11,12
10437,there is indisputable proof that tom is the thief,START_ هناك دليل لا يقبل الجدل أن توم هو اللص ...,9,11
10713,tom came on monday and went back home the foll...,START_ توم أتىَ يوم الإثنين وعاد إلىَ البيت في...,11,12
15341,and adam makary exclaims,START_ ويوضح ادم مكاري _END,4,5
17363,for nostalgic readers the screenshot to the ri...,START_ ويمكن لقراينا الاعزاء الذين يتوقون للما...,18,19
14068,china social media crackdown arrested websites...,START_ حملة ضد مواقع التواصل الاجتماعي في الصي...,9,17
7303,which is heavier lead or gold,START_ أيهما أثقل، الذهب أم الرصاص؟ _END,6,7
20955,you can see a more detailed view of the result...,START_ يمكن ان تشاهد المزيد من التفاصيل هنا مل...,19,17
17221,lebanon facebook group in support of gaza boun...,START_ لبنان مجموعة فيسبوك لمساندة مريم المتجه...,11,12


In [ ]:
lines[lines['length_eng_sentence']<=20]

,English_sentence,Arabic_sentence,length_eng_sentence,length_ara_sentence
19330,exchange student starved in egypt global voices,START_ مصر تجويع طالب تبادل طلابي الاصوات العا...,7,9
11167,iran we re going to the world cup videos globa...,START_ مقاطع فيديو من ايران ذاهبون الى كاس الع...,11,12
10437,there is indisputable proof that tom is the thief,START_ هناك دليل لا يقبل الجدل أن توم هو اللص ...,9,11
10713,tom came on monday and went back home the foll...,START_ توم أتىَ يوم الإثنين وعاد إلىَ البيت في...,11,12
15341,and adam makary exclaims,START_ ويوضح ادم مكاري _END,4,5
...,...,...,...,...
17990,at group blog larvatus prodeo paul norton expl...,START_ وعلى مدونة الجماعية يوضح بول نورتن توجه...,14,13
14748,a fisherman s son tends to play with the tool ...,START_ ابن صياد سمك يلعب بمجسم لقارب _END,13,8
3430,do you really love me,START_ أتحبينني بحق؟ _END,5,4
4532,tom is clearly worn out,START_ توم مرهق بوضوح _END,5,5


### Split the data into train and test

In [ ]:
X, y = lines['English_sentence'], lines['Arabic_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15,random_state=42)
X_train.shape, X_test.shape

((14926,), (2635,))

### Let us save this data

In [ ]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = batch_size):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_eng),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_ara),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_ara, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [ ]:
latent_dim=300
train_samples = len(X_train)
val_samples = len(X_test)

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    7798800     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    17318100    ['input_2[0][0]']                
                                                                                              

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
checkpoint_path = 'best_model.h5'
model_checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True)

In [ ]:
with tf.device("/cpu:0"):
    model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size,
                    callbacks=[early_stopping, model_checkpoint])



Epoch 1/100
233/233 [==============================] - 270s 1s/step - loss: 2.8597 - accuracy: 0.1524 - val_loss: 2.7147 - val_accuracy: 0.1734
Epoch 2/100
233/233 [==============================] - 271s 1s/step - loss: 2.4499 - accuracy: 0.1796 - val_loss: 2.6601 - val_accuracy: 0.1834
Epoch 3/100
233/233 [==============================] - 260s 1s/step - loss: 2.2840 - accuracy: 0.2031 - val_loss: 2.6250 - val_accuracy: 0.2086
Epoch 4/100
233/233 [==============================] - 259s 1s/step - loss: 2.1300 - accuracy: 0.2260 - val_loss: 2.6261 - val_accuracy: 0.2215
Epoch 5/100
233/233 [==============================] - 254s 1s/step - loss: 1.9924 - accuracy: 0.2450 - val_loss: 2.6314 - val_accuracy: 0.2328
Epoch 6/100
233/233 [==============================] - 256s 1s/step - loss: 1.8608 - accuracy: 0.2620 - val_loss: 2.6363 - val_accuracy: 0.2361
Epoch 7/100
233/233 [==============================] - 261s 1s/step - loss: 1.7371 - accuracy: 0.2793 - val_loss: 2.6612 - val_accuracy:

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs2] + decoder_states2)


In [ ]:
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    17318100    ['input_2[0][0]']                
                                                                                                  
 input_3 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 300)]        0           []                               
                                                                                            

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value,verbose=0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [ ]:
scores4grams = 0.0
scores1gram = 0.0
for k in range(50):
    (input_seq, actual_output), _ = next(test_gen)
    decoded_sentence = decode_sequence(input_seq)
    input_english = X_train[k:k+1].values[0]
    actual_ara = y_train[k:k+1].values[0][6:-4]
    pred = decoded_sentence[:-4]
    print('Input English sentence:', input_english)
    print('Actual Arabic Translation:', actual_ara)
    print('Predicted ArabicTranslation:', pred)
    print("######################################################################")
    pred = word_tokenize(pred)
    actual = word_tokenize(actual_ara)
    chencherry = SmoothingFunction()
    score4 = round(sentence_bleu([actual], pred,smoothing_function=chencherry.method1),4)
    print('BLEU SCORE FOR 4-GRAMS:', score4)
    scores4grams+= score4
    print("######################################################################")
    score1 = round(sentence_bleu([actual], pred,(1,0,0,0),smoothing_function=chencherry.method1),4)
    print('BLEU SCORE FOR 1-GRAM:', score1)
    scores1gram+= score1
    print("######################################################################")
    
avg_bleu4 = scores4grams / 50
avg_bleu1 = scores1gram / 50
print("AVERAGE BLEU SCORE FOR 4-GRAMS: ",round(avg_bleu4,4))
print("----------------------------------------------------")
print("AVERAGE BLEU SCORE FOR 1-GRAM: ",round(avg_bleu1,4))

1/1 [==============================] - 0s 22ms/step
Input English sentence: joy and sorrow
Actual Arabic Translation:  الفرح والحزن 
Predicted ArabicTranslation:  نصحها الطبيب بأخذ إجازة 
######################################################################
BLEU SCORE FOR 4-GRAMS: 0
######################################################################
BLEU SCORE FOR 1-GRAM: 0
######################################################################
1/1 [==============================] - 0s 21ms/step
Input English sentence: i am to go there today
Actual Arabic Translation:  سأذهب إلى هناك اليوم 
Predicted ArabicTranslation:  لاحظ يبقى جلب طفل على شرب آخر خمسين صغيرة 
######################################################################
BLEU SCORE FOR 4-GRAMS: 0
######################################################################
BLEU SCORE FOR 1-GRAM: 0
######################################################################
1/1 [==============================] - 0s 23ms/step
Input Engl